# COCO implementation test

This notebook is an implementation of the constrained covariance from [Kernel Constrained Covariance for Dependence Measurement, A.Gretton et al.,2005](https://proceedings.mlr.press/r5/gretton05a/gretton05a.pdf)

### Generating some random variables

In [1]:
import numpy as np

In [36]:
def X(n:int=None,cov: np.array=np.identity(3),mean: np.array = np.zeros(3))->np.array:
    """Args:
    n,int: number of samples by default n=None ie a single sample
    cov,np.array: covariance matrix by default cov=np.identity(3)
    mean: mean value vector by default mean=np.zeros(3)

    returns n samples of the 
    """
    return np.random.multivariate_normal(mean,cov,n)

In [38]:
print(X())

[-1.35713527 -0.82083418 -0.30300738]


### Implementing COCO

We know from (2.3) in the article that: $$COCO_{emp}(z,F,G)=\frac{1}{n}\sqrt{{\lVert \bar{K}^j\bar{K}^g \rVert}_2}$$

With $\bar{K}^f$ the matrix obtained via the projection $ \bar{K}^f  =  PK^fP$ with the projection operator $ P_{ij} = {\delta}_{ij}  - \frac{1}{n}$ and Gram matrix ${K^{f}}_{ij} =k_f(x_i,x_j)$. $\; \bar{K}^g$ is defined analogously.

In [42]:
def map_projection_op(i:int,j:int,n:int)-> float:
    """This function gives the value of Projector operator matrix coefficients
    Args:
        i : line number
        j : row number
        n : number of samples
    Returns:
        P_ij
    """
    if i==j:
        return 1-1/n
    else:
        return -1/n 
    
def map_kernel_gram(i:int,j:int,x:np.array,k:callable) -> float:
    """This function gives the value of the Gram matrix coefficients
    Args:
        i : line number
        j : row number
        x : samples vector
        k : kernel
    Returns:
        K_ij
    """
    return k(x[i],x[j])


def COCO(Kf:callable,Kg:callable,x:np.array)->float:
    """
    """
    n= len(x)
    Kf_gram = np.array([[( map_kernel_gram(i,j,x,Kf) for i in range(n))] for j in range(n)])
    Kg_gram = np.array([[( map_kernel_gram(i,j,x,Kg) for i in range(n))] for j in range(n)])
    P = np.array([[( map_projection_op(i,j,n) for i in range(n))] for j in range(n)])
    Kf_bar = np.matmul(P,np.matmul(Kf_gram,P))
    Kg_bar = np.matmul(P,np.matmul(Kg_gram,P))

    return np.sqrt(np.linalg.norm(np.matmul(Kf_bar,Kg_bar),2))/n
    